In [13]:
import joblib
import pandas as pd

import sys
sys.path.append('..')

In [16]:
# load artifacts (paths — change if needed)
model = joblib.load('../src/diabetes_model.pkl')
scaler = joblib.load('../src/scaler.pkl')
impute_medians = joblib.load('../src/impute_medians.pkl')  # dict {col: median}


In [17]:
feat_order = ['Pregnancies','Glucose','BloodPressure','SkinThickness',
              'Insulin','BMI','DiabetesPedigreeFunction','Age']

In [36]:
# Single patient example (use numbers only)
patient = {
    'Pregnancies': 1,
    'Glucose': 93,    # 0 -> will be imputed
    'BloodPressure': 70,
    'SkinThickness': 31,
    'Insulin': 125,    # 0 -> imputed
    'BMI': 30.4,
    'DiabetesPedigreeFunction': 0.315,
    'Age': 23
}


In [37]:
df_patient = pd.DataFrame([patient])[feat_order]

# impute zeros with saved medians
cols_to_impute = list(impute_medians.keys())
for c in cols_to_impute:
    df_patient[c] = df_patient[c].replace(0, impute_medians[c])

# ensure numeric (optional)
df_patient = df_patient.astype(float)

# scale
X_patient = scaler.transform(df_patient)  # scaler fitted on training features in same order

# predict
probs = model.predict_proba(X_patient)[:, 1]
preds = model.predict(X_patient)

# show results
for i, (p, pr) in enumerate(zip(preds, probs)):
    print(f"Row {i}: Predicted class = {int(p)}, Probability = {pr:.4f}")

Row 0: Predicted class = 0, Probability = 0.0561
